In [3]:
# Import all libraries needed for the tutorial

# General syntax to import specific functions in a library: 
##from (library) import (specific library function)
from pandas import DataFrame, read_csv

# General syntax to import a library but no functions: 
##import (library) as (give the library a nickname/alias)
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

# Enable inline plotting
%matplotlib inline

import scipy
import numpy as np
import scipy.signal as signal
from scipy.signal import *
import matplotlib.pyplot as plt

%matplotlib tk

In [4]:
#load data
# file path of csv file
#Location = r'/Users/33614/ExternalDrive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/SYL21/D_Drive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/louiseplacidet/Desktop/PIR/Data/data_spikes/E18KABaseline_Bcut.txt'
Location = r'/Users/louiseplacidet/Desktop/PIR/Data/new_spike_data/newdata/E18KABaseline_BcutV2groundAll.txt'


# create dataframe
df = pd.read_csv(Location, sep='\t',skiprows=[0,1,3] , index_col='%t           ')

In [5]:
#####################################################################################################################
####  BANK OF PARTS OF DATA

all_raw_data = df #Entire recording from all electrodes

full_signal = df.iloc[:,1] #Entire recording from the first electrode

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

# Sample rate and desired cutoff frequencies (in Hz).
fs = 25000.0
lowcut = 100.0
highcut = 2500.0


y = butter_bandpass_filter(df.iloc[:,1], lowcut, highcut, fs, order=6)


filtereddf = pd.DataFrame(y)
filtereddf.index = df.index

signal_filtered = filtereddf.iloc[:,0] #Entire recording filtered by bandpass, for one electrode


fs = 25000

xminnoise = int(np.round(11114*(fs/1000)))
xmaxnoise = int(np.round(18511*(fs/1000)))

noise_data = filtereddf.iloc[xminnoise:xmaxnoise,0]

xminspike = int(np.round(130826*(fs/1000)))
xmaxspike = int(np.round(131699*(fs/1000)))

burst_data = filtereddf.iloc[xminspike:xmaxspike,0]


#############################################

y_ref = butter_bandpass_filter(df['El 15       '], lowcut, highcut, fs, order=6)

filtereddf_ref = pd.DataFrame(y_ref)
filtereddf_ref.index = df.index

In [6]:
#threshold

def test_valid_window(window, test_level = 5):
    """
    window : the window in the signal that has to be tested
    
    This funtion test the window to insure that it doesn't contain the signal of interest (spike)
    """
    #non zero ?
    second = np.percentile(window, 2)
    thirtyth = np.percentile(window, 30)
    #print(str(second) + "\t" + str(thirtyth) + "\t" + str(second/thirtyth))
    if abs(second/thirtyth) < test_level : 
        return True
    else : 
        return False
    
def init_noise_levels(signal, fs, 
                      noise_window_size = 0.01,
                      required_valid_windows = 100,
                      old_noise_level_propagation = 0.8, 
                      test_level = 5,
                      estimator_type = "RMS",
                      percentile_value = 25):
    
    if estimator_type == "RMS":
        return init_noise_levels_RMS(signal, fs, 
                      noise_window_size = noise_window_size,
                      required_valid_windows = required_valid_windows,
                      old_noise_level_propagation = old_noise_level_propagation, 
                      test_level = test_level,
                      percentile_value = percentile_value)
        
    elif estimator_type == "MAD":
        return init_noise_levels_MAD(signal, fs, 
                      noise_window_size = noise_window_size,
                      required_valid_windows = required_valid_windows,
                      old_noise_level_propagation = old_noise_level_propagation, 
                      test_level = test_level,
                      percentile_value = percentile_value)
    
    else: return None
    
    
def init_noise_levels_RMS(signal, fs, 
                      noise_window_size = 0.01,
                      required_valid_windows = 100,
                      old_noise_level_propagation = 0.8, 
                      test_level = 5,
                      percentile_value = 25):
    
    nb_valid_windows = 0
    list_RMS = []
    noise_levels = []
    
    noise_level = -1
    
     
    #boucle en indice
#    for window_index in range(0,len(signal)-(len(signal)%int(fs*noise_window_size)),int(fs*noise_window_size)):
    for window_index in range(0,len(signal),int(fs*noise_window_size)):
        test = test_valid_window(signal.iloc[window_index: window_index + int(fs*noise_window_size)], test_level)
        if nb_valid_windows < required_valid_windows:
            if test == True :
                RMS = np.sqrt(np.mean(signal.iloc[window_index: window_index + int(fs*noise_window_size)]**2))
                list_RMS.append(RMS)
                nb_valid_windows += 1
            
            if nb_valid_windows == required_valid_windows:
                noise_level = np.percentile(list_RMS, percentile_value)
                for elm in range(0, window_index, int(fs*noise_window_size)):
                    noise_levels.append(noise_level)
                
        else :
            """if test == True:
                if (window + int(fs*noise_window_size)) > (len(signal)-1) :
                    N25 = np.percentile(abs(signal.iloc[window:]), 25)
                else :
                    N25 = np.percentile(abs(signal.iloc[window: window + int(fs*noise_window_size)]), 25)
                noise_level = old_noise_level_propagation*noise_level + (1-old_noise_level_propagation)*N25
            noise_levels.append(noise_level)"""
            if test == True:
                if (window_index + int(fs*noise_window_size)) > (len(signal)-1) :
                    RMS = np.sqrt(np.mean(signal.iloc[window_index:]**2))
                else :
                    RMS = np.sqrt(np.mean(signal.iloc[window_index: window_index + int(fs*noise_window_size)]**2))
                list_RMS.append(RMS)
                NX = np.percentile(list_RMS, percentile_value)
                new_noise_level = old_noise_level_propagation*noise_level + (1-old_noise_level_propagation)*NX
                noise_level = new_noise_level
            noise_levels.append(noise_level)
            
    #cas ou il n'y a pas eut 100 fenetres de bruit valides rencontrees
    if noise_level == -1:
        
        #cas ou aucune fenetre valide n'a ete rencontree
        if noise_levels == []:
            for elm in range(0, len(signal), int(fs*noise_window_size)):
                noise_levels.append(0)
            
        else:
            noise_level = np.percentile(list_RMS, percentile_value)
            for elm in range(0, len(signal), int(fs*noise_window_size)):
                noise_levels.append(noise_level)
        
    
    noise_levels.append(noise_level)        
    plt.figure()
    plt.plot(list_RMS)
    plt.xlabel('Time Windows')
    plt.title('RMS values')
    plt.grid(True)
                
    return noise_levels

def init_noise_levels_MAD(signal, fs, 
                      noise_window_size = 0.01,
                      required_valid_windows = 100,
                      old_noise_level_propagation = 0.8, 
                      test_level = 5,
                      percentile_value = 25):
    
    nb_valid_windows = 0
    list_MAD = []
    noise_levels = []
     
    #boucle en indice
    for window_index in range(0,len(signal),int(fs*noise_window_size)):
        test = test_valid_window(signal.iloc[window_index: window_index + int(fs*noise_window_size)], test_level)
        if nb_valid_windows < required_valid_windows:
            if test == True :
                ###RMS = np.sqrt(np.mean(signal.iloc[window_index: window_index + int(fs*noise_window_size)]**2))
                MAD = scipy.stats.median_absolute_deviation(signal.iloc[window_index: window_index + int(fs*noise_window_size)])
                list_MAD.append(MAD)
                nb_valid_windows += 1
            
            if nb_valid_windows == required_valid_windows:
                noise_level = np.percentile(list_MAD, percentile_value)
                for elm in range(0, window_index, int(fs*noise_window_size)):
                    noise_levels.append(noise_level)
                
        else :
            """if test == True:
                if (window + int(fs*noise_window_size)) > (len(signal)-1) :
                    N25 = np.percentile(abs(signal.iloc[window:]), 25)
                else :
                    N25 = np.percentile(abs(signal.iloc[window: window + int(fs*noise_window_size)]), 25)
                noise_level = old_noise_level_propagation*noise_level + (1-old_noise_level_propagation)*N25
            noise_levels.append(noise_level)"""
            if test == True:
                if (window_index + int(fs*noise_window_size)) > (len(signal)-1) :
                    ###RMS = np.sqrt(np.mean(signal.iloc[window_index:]**2))
                    MAD = scipy.stats.median_absolute_deviation(signal.iloc[window_index:])
                else :
                    ###RMS = np.sqrt(np.mean(signal.iloc[window_index: window_index + int(fs*noise_window_size)]**2))
                    MAD = scipy.stats.median_absolute_deviation(signal.iloc[window_index: window_index + int(fs*noise_window_size)])
                list_MAD.append(MAD)
                NX = np.percentile(list_MAD, percentile_value)
                new_noise_level = old_noise_level_propagation*noise_level + (1-old_noise_level_propagation)*NX
                noise_level = new_noise_level
            noise_levels.append(noise_level)
    
    noise_levels.append(noise_levels)        
    plt.figure()
    plt.plot(list_MAD)
    plt.xlabel('Time Windows')
    plt.title('MAD values')
    plt.grid(True)
                
    return noise_levels

In [7]:
#find spike

def find_spike(signal, initial_index, noise_levels, fs, spike_centers, 
               window_size = 0.001, 
               noise_window_size = 0.01,
               threshold_factor = 4):
    
    offset_index = int(np.round(signal.index[0]*fs/1000))
    
    if initial_index < len(signal) + offset_index:
        i = initial_index
        for value in signal.iloc[initial_index-offset_index:]:
            threshold = threshold_factor*noise_levels[int(np.round((i/fs)//noise_window_size))]
            if value > threshold or value < -threshold:
                while(True):
                    if i > initial_index + window_size*fs:
                        b_point = int(np.round(i - window_size*fs)) - offset_index
                    else :
                        b_point = initial_index - offset_index
                    if i < len(signal)+offset_index - window_size*fs-1:
                        e_point = int(np.round(i + window_size*fs)) - offset_index
                    else :
                        e_point = len(signal)

                    highest_value = signal.iloc[b_point: e_point].max()
                    lowest_value = signal.iloc[b_point: e_point].min()
                    
                    if abs(highest_value) > abs(lowest_value):
                        extremum_value = highest_value
                    else : extremum_value = lowest_value
                    #print("highest value:"+str(highest_value))
                    
                    if extremum_value == value : # && 50% amplitude
                        #print("Found high spike")
                        #print("index:"+str(i))
                        spike_centers.append(i)
                        return (e_point+offset_index)
                    
                    else:
                        #i = signal.index.get_loc(highest)
                        #print("b_point high: " + str(b_point))
                        #print("e_point high: " + str(e_point))
                        if abs(highest_value) > abs(lowest_value):
                            i = int(np.round(signal.iloc[b_point: e_point].idxmax()*fs/1000))
                        else : 
                            #print(signal.iloc[b_point:e_point])
                            i = int(np.round(signal.iloc[b_point: e_point].idxmin()*fs/1000))
                        #print("i high: "+str(i))
                        #print("value: "+str(signal.iloc[i]))
                        value = signal.iloc[i-offset_index]
                        
                break                  
            i += 1
    return -2
            
def find_spikes(signal, noise_levels, fs, 
               window_size = 0.001, 
               noise_window_size = 0.01,
               threshold_factor = 4):
    
    initial = int(np.round(signal.index[0]*fs/1000))
    spike_centers = []
    
    while initial != -2:
        initial = find_spike(signal, initial, noise_levels, fs, spike_centers,
                             window_size = window_size, 
                             noise_window_size = noise_window_size,
                             threshold_factor = threshold_factor)
    
    return spike_centers

In [19]:
# record spike

def record_spikes(signal, fs, spike_info,
                  align_method,
                  t_before = 0.001,
                  t_after = 0.002):
    
    if (align_method in spike_info.columns) == False:
        print("align_method is incorrect, please choose one of the following :" + str(spike_info.columns))
        return None
    
    else:
        spike_centers = spike_info[align_method].values
        
    t_b = int(np.round(fs*(t_before)))
    t_a = int(np.round(fs*(t_after)))
    
    data = np.array([[float(x) for x in range(t_b+t_a+1)]])
    
    initial_index = int(np.round(signal.index[0]*fs/1000))
    
    for center in spike_centers:
        if center < t_b + initial_index:
            spike = [0 for i in range(0, t_b-(center-initial_index))]
            spike = np.concatenate((spike, signal.values[:center + t_a - initial_index]))
            data = np.insert(data, len(data), spike, axis=0)
            
        elif center > len(signal)-t_a + initial_index:
            spike = signal.values[center - t_b - initial_index:]
            spike = np.concatenate((spike,[0 for i in range(0, t_a - (len(signal) + initial_index-center))]))
            data = np.insert(data, len(data), spike, axis=0)
            
        else :
            spike = signal.values[center - t_b - initial_index: center + t_a + 1 - initial_index]
            data = np.insert(data, len(data), spike, axis=0)

    print(np.shape(data))
    data = data.transpose()
    spike_data = pd.DataFrame(data)
    
    return spike_data

In [20]:
def record_spikes_oneline(signal, fs, spike_info,
                  align_method,
                  t_before = 0.001,
                  t_after = 0.002):
    
    if (align_method in spike_info.columns) == False:
        print("align_method is incorrect, please choose one of the following :" + str(spike_info.columns))
        return None
    
    else:
        spike_centers = spike_info[align_method].values
        
    t_b = int(np.round(fs*(t_before)))
    t_a = int(np.round(fs*(t_after)))
    
    data = np.array(['NaN' for x in range(len(signal))])
    data = data.astype(float)
    times = np.array(['NaN' for x in range(len(signal))])
    times = times.astype(pd.Timestamp)
    
    for center in spike_centers:
        if center < t_b:
            data[:center + t_a] = signal.values[:center + t_a]
            times[:center + t_a] = signal.index[:center + t_a]
            
        elif center > len(signal) - t_a:
            data[center - t_b:] = signal.values[center - t_b:]
            times[center - t_b:] = signal.index[center - t_b:]
            
        else :
            data[center - t_b: center + t_a + 1] = signal.values[center - t_b: center + t_a + 1]
            times[center - t_b: center + t_a + 1] = signal.index[center - t_b: center + t_a + 1]

    spike_data_oneline = pd.DataFrame(data, index = times.astype(float))
    
    return spike_data_oneline

In [21]:
################################################################################################
####   TEST ADABANDFLT AVEC SIGNAL ORIGINAL (PASSE-BANDE+WIENER)

# Choices:
#  - full_signal : entire signal from first electrode
#  - signal_filtered : entire signal from first electrode after bandpass
#  - noise_data : part of signal where only noise (after bandpass)
#  - spike_data : part of signal where burst (after bandpass)


signal = signal_filtered
signal_filtered_ref = filtereddf_ref.iloc[:,0] #Entire recording filtered by bandpass, for reference electrode

In [22]:
figure = plt.figure()
plt.gcf().subplots_adjust(left = 0.1, bottom = 0.1, right = 0.9, top = 0.9, wspace = 0.2, hspace = 0.5)
x = signal_filtered_ref.values
axes = figure.add_subplot(2, 1, 1)
axes.plot(signal_filtered_ref.index, x)
axes.set_xlabel('Time in ms')
axes.set_title('Signal ref')
axes.set_ylim(-10 , 10)
axes.grid(True)
x = signal_filtered.values
axes = figure.add_subplot(2, 1, 2)
axes.plot(signal_filtered.index, x)
axes.set_xlabel('Time in ms')
axes.set_title('Signal 31')
axes.set_ylim(-10 , 10)
axes.grid(True)

In [23]:
fs = 25000

noise_levels = init_noise_levels(signal_filtered_ref, fs, 
                      noise_window_size = 0.01,
                      required_valid_windows = 100,
                      old_noise_level_propagation = 0.8, 
                      test_level = 5,
                      estimator_type = "RMS",
                      percentile_value = 25)


In [24]:
plt.figure()
plt.plot(noise_levels)
plt.grid(True)
plt.xlabel('Time')
plt.ylabel('Noise Amplitude [µV]')
plt.title('Noise Levels')

Text(0.5, 1.0, 'Noise Levels')

In [25]:
spike_centers = find_spikes(signal, noise_levels, fs, 
                           window_size = 0.001, 
                           noise_window_size = 0.01,
                           threshold_factor = 4)

spike_centers

[18,
 112,
 172,
 277,
 438,
 481,
 597,
 759,
 808,
 973,
 1119,
 1163,
 1253,
 1399,
 1498,
 1603,
 1637,
 1697,
 1725,
 1775,
 1824,
 1958,
 2091,
 2176,
 2241,
 2306,
 2342,
 2396,
 2545,
 2624,
 2904,
 3022,
 3059,
 3198,
 3265,
 3312,
 3377,
 3405,
 3571,
 3748,
 3854,
 3886,
 3997,
 4022,
 4100,
 4328,
 4389,
 4512,
 4541,
 4637,
 4825,
 4895,
 4956,
 5056,
 5140,
 5194,
 5232,
 5332,
 5424,
 5497,
 5545,
 5632,
 5668,
 5710,
 5812,
 5853,
 5902,
 5946,
 5976,
 6040,
 6097,
 6193,
 6239,
 6267,
 6331,
 6364,
 6641,
 6689,
 6725,
 6800,
 6867,
 7083,
 7125,
 7153,
 7241,
 7332,
 7540,
 7579,
 7665,
 7906,
 8067,
 8168,
 8318,
 8364,
 8514,
 8539,
 8597,
 8724,
 8797,
 8838,
 8884,
 8979,
 9078,
 9164,
 9287,
 9361,
 9399,
 9437,
 9563,
 9632,
 9741,
 9781,
 9846,
 9998,
 10067,
 10103,
 10196,
 10261,
 10345,
 10402,
 10475,
 10640,
 10921,
 11002,
 11055,
 11164,
 11214,
 11249,
 11292,
 11318,
 11387,
 11435,
 11468,
 11587,
 11696,
 11951,
 12010,
 12046,
 12151,
 12203,
 1225

In [26]:
len(spike_centers)

6700

In [27]:
spike_data = record_spikes(signal, fs, spike_centers, 
                  t_before = 0.0008, 
                  t_after = 0.0008)

TypeError: record_spikes() missing 1 required positional argument: 'align_method'

In [ ]:
spike_data_oneline = record_spikes_oneline(signal, fs, spike_centers,
                  t_before = 0.0008,
                  t_after = 0.0008)

In [ ]:
plt.figure(figsize = (100,20))
#plt.plot(spike_data)
spike_data.iloc[:,1:200].plot()
plt.xlabel('Time')
plt.grid()

In [34]:
#signal.plot(color = 'blue')
#spike_data_oneline.plot(color = 'red')
plt.plot(df.index, signal, color = 'blue')
plt.plot(spike_data_oneline.index, spike_data_oneline, color = 'red')
plt.title('Filtered Signal with Detected Spikes with RMS')
plt.xlabel('Time Windows')
plt.ylabel('Amplitude [µV]')
plt.legend()
plt.grid(True)

/opt/anaconda3/lib/python3.7/tkinter/__init__.py:749: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)
/opt/anaconda3/lib/python3.7/tkinter/__init__.py:1705: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  return self.func(*args)
